## Zentralitätsmetriken

PageRank: Identifiziert die Einflussreichsten Knoten basierend darauf, wie viele Verbindungen (Kanten) auf sie zeigen und wie einflussreich die Knoten sind, die auf sie zeigen. <br>
Betweeness Centralitiy: Misst, wie oft ein Knoten auf dem kürzesten Pfad zwischen anderen Knotenpaaren liegt. Hohe Betweenness-Werte weisen auf Knoten hin, die als wichtige Vermittler innerhalb des Netzwerks fungieren 

#### Cypher Pagerank Query

In [ ]:
from py2neo import Graph

graph = Graph("bolt://localhost:7687", auth=("neo4j", ""))

pagerank_query = """
CALL gds.pageRank.stream('myGraphConnections')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC
"""
pagerank_results = graph.run(pagerank_query).data()

for result in pagerank_results:
    print(result)

Output Deutung: <br>
Zeigt die "Wichtigkeit" oder den Einfluss jedes Knotens (repräsentiert durch Personen) in Ihrem Netzwerk, basierend auf der PageRank-Metrik. <br>

James Becker: hat den höchsten PageRank-Score, was darauf hindeutet, dass er im Zentrum des Netzwerks steht. Dies bedeutet, dass James direkt oder indirekt gut mit anderen wichtigen Knoten verbunden ist.<br>
Ava Weber und Oliver Schulz: erscheinen auch mit hohen Scores, was auf ihre zentralen Rollen im Netzwerk hinweist. <br>
John Smith, Max Mustermann und Jane Doe: haben die niedrigsten Scores, was darauf hinweisen könnte, dass sie weniger zentral sind oder weniger direkte Verbindungen zu anderen wichtigen Knoten haben.

#### Übergabe an GPT

In [ ]:
import openai

class OpenAIWrapper:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = self.api_key

    def generate_chat_response(self, prompt, model="gpt-3.5-turbo", temperature=0.7, max_tokens=150):
        try:
            response = openai.ChatCompletion.create(
                model=model,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                temperature=temperature,
                max_tokens=max_tokens
            )
            return response.choices[0].message['content'].strip() 
        except Exception as e:
            return f"An error occurred: {str(e)}"

In [ ]:
openai_wrapper = OpenAIWrapper(api_key='')

prompt_intro = "Based on the PageRank analysis of the network, the nodes are ranked according to their influence within the network. The PageRank score indicates the likelihood of arriving at a node through random walks. Here are the top-ranked nodes based on their PageRank scores:\n\n"

prompt_body = "\n".join([f"- {res['name']} with a PageRank score of {res['score']:.2f}" for res in pagerank_results[:10]])  # Top 10 für den Prompt

prompt_question = "\n\nWhat insights can be drawn from these PageRank scores regarding the structure and influence dynamics within the network? Additionally, what further analyses could provide deeper understanding of the network's characteristics?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")

#### Cypher Betweenness Query

In [ ]:
betweenness_query = """
CALL gds.betweenness.stream('myGraphConnections')
YIELD nodeId, score
RETURN gds.util.asNode(nodeId).name AS name, score
ORDER BY score DESC
"""

betweenness_results = graph.run(betweenness_query).data()

for result in betweenness_results:
    print(result)

Output Deutung: <br>
James Becker und Oliver Schulz zeigen sehr hohe Betweenness-Werte, was darauf hindeutet, dass sie wichtige Verbindungspunkte im Netzwerk sind <br>
Max Mustermann, Jane Doe, und John Smith haben eine Betweenness Centrality von 0, was bedeutet, dass sie nicht auf kürzesten Pfaden zwischen anderen Knoten liegen. 

#### Übergabe an GPT

In [ ]:
prompt_intro = "Based on the Betweenness Centrality analysis of the network, the nodes are ranked according to their roles as intermediaries in the communication paths across the network. The Betweenness Centrality score indicates how often a node acts as a bridge along the shortest path between two other nodes. Here are the nodes with the highest Betweenness Centrality scores:\n\n"

prompt_body = "\n".join([f"- {res['name']} with a Betweenness Centrality score of {res['score']:.2f}" for res in betweenness_results[:10]])  # Nehmen wir die Top 10 für den Prompt

prompt_question = "\n\nWhat can be inferred about the network's structure and the importance of these nodes from these Betweenness Centrality scores? What implications do these scores have for the flow of information or influence within the network?"

complete_prompt = prompt_intro + prompt_body + prompt_question

print(complete_prompt)

response = openai_wrapper.generate_chat_response(complete_prompt)
print(f"GPT-3's Insights: {response}")